In [1]:
import pymongo
from bson import ObjectId
from pprint import pprint as pp
import pandas as pd
import re
from collections import defaultdict

In [2]:
data = pd.read_csv("/home/tony.hurkovskyi/Documents/tickets/PBMP-3121/advanced_export2.csv")

/home/tony.hurkovskyi/.virtualenvs/datarobot-7.0/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,21,22,286) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
conn = pymongo.MongoClient('shrink-mongo-0.infra.ent.datarobot.com', 27017)
db = conn['MMApp']
# pregex for retrieving FEAR we don't want to have same extraction many times (1st lag, 2nd lag)
# so, we treat it as the same extraction
REGEXP = re.compile(r'\((?:\d+\w+ )?((?:[a-zA-Z]+ )*[a-zA-Z]+)\)')

## Get all models from liderboards of overall projects in the MBTest

In [4]:
models_query = {'pid': {'$in': [ObjectId(pid) for pid in set(data.pid.values)]}}
models_cursor = db.leaderboard.find(models_query)
models = [model for model in models_cursor]

## Get all projects from the overall projects in the MBTest

In [5]:
projects_cursor = db.project.find({'_id': models_query['pid']})
projects = [project for project in projects_cursor]

### Create Pandas series for Projects

In [6]:
project_dict = {}
for project in projects:
    for k in project:
        project_dict[k] = []
for i, project in enumerate(projects):
    for k in project_dict:
        if k in project:
            project_dict[k].append(project[k])
        else:
            project_dict[k].append(None)
project_pd = pd.DataFrame.from_dict(data=project_dict)
project_pd['_id'] = project_pd['_id'].apply(lambda i: str(i))
project_pd.head()

,_id,accuracy_optimized_mb,active,autopilot_blend_best_models,autopilot_consider_blenders_in_recommendation,autopilot_prepare_model_for_deployment,autopilot_run_leakage_removed_feature_list,autopilot_scoring_code_only,created,default_dataset_id,...,status_traceback,target,target_options,test_dataset_metric_detail,tokens,uid,unsupervised_mode,version,weights,workers
0,6014455316330abc32bae69a,False,1,True,False,True,True,False,1.611941e+09,601445d08067262630c27f96,...,,"{u'type': u'Regression', u'name': u'y (actual)...","{u'positive_class': None, u'missing_maps_to': ...",None,{u'60144552ae3955fab1fbc1d8': u'RNhJ_EjOaXKObA...,60144552ae3955fab1fbc1d8,None,1.8,"{u'partition': [u'partition_col'], u'sample_we...",20
1,60144554783ba7b75ebae69a,False,1,True,False,True,False,False,1.611941e+09,60144625220d82d44014b76b,...,,"{u'type': u'Regression', u'name': u'DR_FAKE_TA...","{u'positive_class': None, u'missing_maps_to': ...","[{u'name': u'AUC', u'ascending': True}, {u'nam...",{u'60144552ae3955fab1fbc1d8': u'IBEXUKELbvFbdg...,60144552ae3955fab1fbc1d8,"{u'enabled': True, u'fake_target_name': u'DR_F...",1.8,"{u'partition': [u'partition_col'], u'sample_we...",20
2,6014455516330abc32bae6a0,False,1,True,False,True,False,False,1.611941e+09,60144893d1b4f779307461c0,...,,"{u'type': u'Regression', u'name': u'DR_FAKE_TA...","{u'positive_class': None, u'missing_maps_to': ...","[{u'name': u'AUC', u'ascending': True}, {u'nam...",{u'60144552ae3955fab1fbc1d8': u'gupMMI-XJ4alSQ...,60144552ae3955fab1fbc1d8,"{u'enabled': True, u'fake_target_name': u'DR_F...",1.8,"{u'partition': [u'partition_col'], u'sample_we...",20
3,6014461f16330abc32bae6f8,False,1,True,False,True,False,False,1.611941e+09,6014473c0f15c5fb3c14b71d,...,,"{u'type': u'Regression', u'name': u'DR_FAKE_TA...","{u'positive_class': None, u'missing_maps_to': ...","[{u'name': u'AUC', u'ascending': True}, {u'nam...",{u'60144552ae3955fab1fbc1d8': u'VgdAApwAgA_itg...,60144552ae3955fab1fbc1d8,"{u'enabled': True, u'fake_target_name': u'DR_F...",1.8,"{u'partition': [u'partition_col'], u'sample_we...",20
4,6014469c16330abc32bae701,False,1,True,False,True,True,False,1.611942e+09,6014473136335d6567c27f3d,...,,"{u'type': u'Regression', u'name': u'sales_qty_...","{u'positive_class': None, u'missing_maps_to': ...",None,{u'60144552ae3955fab1fbc1d8': u'vqn-LFTeC9d2NQ...,60144552ae3955fab1fbc1d8,None,1.8,"{u'partition': [u'partition_col'], u'sample_we...",20


### Create Pandas Series for Models adding the extra column for sorting

In [7]:
models_dict = {'sorting_metrics': []}
for model in models:
    for k in model:
        models_dict[k] = []
for i, model in enumerate(models):
    for k in models_dict:
        if k in model:
            models_dict[k].append(model[k])
        elif k == 'sorting_metrics':
            models_dict[k].append(model['test'][project_pd.loc[project_pd._id == str(model['pid'])].metric.values[0]][-1])
        else:
            models_dict[k].append(None)
    
models_pd = pd.DataFrame.from_dict(data=models_dict)
models_pd.head()

,_id,all_best_parameters,all_parameters,autopilot_run_ids,backtest_training_info,best_parameters,blend,blender,blueprint,blueprint_id,...,training_dataset_name,training_length,uid,use_codegen,use_gpu,use_kubeworkers,vcores,vertex_cnt,vertex_files,vertices
0,60144619d246b440b7c27f05,"{u'(0, -1)': {u'7': {u'FD1_enet_lambda': 0.316...","{u'(0, -1)': [[u'FD1_a', u'FD1_enet_alpha', u'...",[601445e4d246b440b7c27ebc],"{u'1': {u'status': u'NOT_COMPLETED', u'observe...","{u'(0, -1)': {u'FD1_enet_lambda': 0.3162277660...",0,None,"{u'1': [[u'CAT'], [u'PDM3 cm=50000;dtype=float...",02ba273c63e2ff134b5701c1d79291ac,...,default,P10Y11M1D,60144552ae3955fab1fbc1d8,None,False,False,4,7,"{u'(0, -1)': {u'38fa14c4c59c347b658207418d5226...",{}
1,60146088573e3d61a8c27f14,"{u'(0, -1)': {}, u'(2, -1)': {}, u'(1, -1)': {...","{u'(0, -1)': [], u'(2, -1)': [], u'(1, -1)': [...",[601445e4d246b440b7c27ebc],"{u'1': {u'status': u'NOT_COMPLETED', u'observe...",{},0,"{u'inputs': [{u'blueprint': {u'1': [[u'CAT'], ...","{u'1': [[u'd9fcdb969ea59abed073927d9568730a'],...",2cbe880e11f0d874e8965e1ba9eb912e,...,default,P10Y11M1D,60144552ae3955fab1fbc1d8,None,False,False,4,1,"{u'(0, -1)': {u'89222f9e9f9bb3a811b8c769d55f6d...",{}
2,60144619d246b440b7c27f01,"{u'(0, -1)': {u'5': {u'FD1_enet_lambda': 0.316...","{u'(0, -1)': [[u'FD1_a', u'FD1_enet_alpha', u'...",[601445e4d246b440b7c27ebc],"{u'1': {u'status': u'NOT_COMPLETED', u'observe...","{u'(0, -1)': {u'FD1_enet_lambda': 0.3162277660...",0,None,"{u'1': [[u'CAT'], [u'PDM3 cm=50000;dtype=float...",31762438654a64f68649cce8f4958568,...,default,P10Y11M1D,60144552ae3955fab1fbc1d8,None,False,False,4,5,"{u'(0, -1)': {u'89222f9e9f9bb3a811b8c769d55f6d...",{}
3,60145b10498bf49406746173,"{u'(0, -1)': {u'5': {u'FD1_enet_lambda': 0.316...","{u'(0, -1)': [[u'FD1_a', u'FD1_enet_alpha', u'...",[601445e4d246b440b7c27ebc],"{u'1': {u'status': u'NOT_COMPLETED', u'observe...","{u'(0, -1)': {u'FD1_enet_lambda': 0.3162277660...",0,None,"{u'1': [[u'CAT'], [u'PDM3 cm=50000;dtype=float...",31762438654a64f68649cce8f4958568,...,default,P10Y11M1D,60144552ae3955fab1fbc1d8,None,False,False,4,5,"{u'(0, -1)': {u'1297352fb3e5e9f196a9f975b24e49...",{}
4,60144618d246b440b7c27efe,"{u'(0, -1)': {u'10': {u'num_leaves': 16, u'reg...","{u'(0, -1)': [[u'cbt', u'colsample_bytree', u'...",[601445e4d246b440b7c27ebc],"{u'1': {u'status': u'NOT_COMPLETED', u'observe...","{u'(0, -1)': {u'num_leaves': 16, u'reg_alpha':...",0,None,"{u'10': [[u'7', u'9'], [u'RES_ESLGBMTR lr=0.05...",34d8bdc3e8848487792846da4b286f9f,...,default,P10Y11M1D,60144552ae3955fab1fbc1d8,None,False,False,4,10,"{u'(0, -1)': {u'94bc40638a07fb843e93bbdeded06d...",{}


In [8]:
sorted_models = models_pd.sort_values(by=['pid', 'sorting_metrics'])
sorted_models.head()

,_id,all_best_parameters,all_parameters,autopilot_run_ids,backtest_training_info,best_parameters,blend,blender,blueprint,blueprint_id,...,training_dataset_name,training_length,uid,use_codegen,use_gpu,use_kubeworkers,vcores,vertex_cnt,vertex_files,vertices
28,60145fdb7cfd2f6383746194,"{u'(-1, -1)': {u'5': {u'FD1_enet_lambda': 0.31...","{u'(-1, -1)': [[u'FD1_a', u'FD1_enet_alpha', u...",[601445e4d246b440b7c27ebc],{u'1': {u'status': u'BACKTEST_BOUNDARIES_EXCEE...,"{u'(-1, -1)': {u'FD1_fit_alpha_scaler': True, ...",0,None,"{u'1': [[u'CAT'], [u'PDM3 cm=50000;dtype=float...",f9192436c7a7a7d61a4b603793a3be74,...,default,2004-10-29T00:00:00.000000Z/2015-10-01T00:00:0...,60144552ae3955fab1fbc1d8,None,False,False,4,5,"{u'(-1, -1)': {u'5053e80b3c121767a3ef47b39bfb9...",{}
3,60145b10498bf49406746173,"{u'(0, -1)': {u'5': {u'FD1_enet_lambda': 0.316...","{u'(0, -1)': [[u'FD1_a', u'FD1_enet_alpha', u'...",[601445e4d246b440b7c27ebc],"{u'1': {u'status': u'NOT_COMPLETED', u'observe...","{u'(0, -1)': {u'FD1_enet_lambda': 0.3162277660...",0,None,"{u'1': [[u'CAT'], [u'PDM3 cm=50000;dtype=float...",31762438654a64f68649cce8f4958568,...,default,P10Y11M1D,60144552ae3955fab1fbc1d8,None,False,False,4,5,"{u'(0, -1)': {u'1297352fb3e5e9f196a9f975b24e49...",{}
1,60146088573e3d61a8c27f14,"{u'(0, -1)': {}, u'(2, -1)': {}, u'(1, -1)': {...","{u'(0, -1)': [], u'(2, -1)': [], u'(1, -1)': [...",[601445e4d246b440b7c27ebc],"{u'1': {u'status': u'NOT_COMPLETED', u'observe...",{},0,"{u'inputs': [{u'blueprint': {u'1': [[u'CAT'], ...","{u'1': [[u'd9fcdb969ea59abed073927d9568730a'],...",2cbe880e11f0d874e8965e1ba9eb912e,...,default,P10Y11M1D,60144552ae3955fab1fbc1d8,None,False,False,4,1,"{u'(0, -1)': {u'89222f9e9f9bb3a811b8c769d55f6d...",{}
2,60144619d246b440b7c27f01,"{u'(0, -1)': {u'5': {u'FD1_enet_lambda': 0.316...","{u'(0, -1)': [[u'FD1_a', u'FD1_enet_alpha', u'...",[601445e4d246b440b7c27ebc],"{u'1': {u'status': u'NOT_COMPLETED', u'observe...","{u'(0, -1)': {u'FD1_enet_lambda': 0.3162277660...",0,None,"{u'1': [[u'CAT'], [u'PDM3 cm=50000;dtype=float...",31762438654a64f68649cce8f4958568,...,default,P10Y11M1D,60144552ae3955fab1fbc1d8,None,False,False,4,5,"{u'(0, -1)': {u'89222f9e9f9bb3a811b8c769d55f6d...",{}
26,60144619d246b440b7c27f04,"{u'(0, -1)': {u'10': {u'num_leaves': 16, u'reg...","{u'(0, -1)': [[u'cbt', u'colsample_bytree', u'...",[601445e4d246b440b7c27ebc],"{u'1': {u'status': u'NOT_COMPLETED', u'observe...","{u'(0, -1)': {u'num_leaves': 16, u'reg_alpha':...",0,None,"{u'10': [[u'7', u'9'], [u'RES_ESLGBMTR lr=0.05...",e5c0a5aea5b6545a0b0466f8a54f5e41,...,default,P10Y11M1D,60144552ae3955fab1fbc1d8,None,False,False,4,10,"{u'(0, -1)': {u'70913453ea6e1dba429cdc81937319...",{}


## Get all metadata about data from MBTests

In [9]:
dataset_ids = {m['dataset_id'] for m in models}
dataset_query = {'_id': {'$in': [ObjectId(i) for i in dataset_ids]}}
dataset_cursor = db.metadata.find(dataset_query)
datasets = [dataset for dataset in dataset_cursor]

### Extend metadata with ALL_CNT (all vars in featurelist taking into account FEAR) & FEAR (specific extraction) & FEAR_CNT (count of this extraction) info

In [10]:
f_datasets = []
for dataset in datasets:
    fear = defaultdict(int)
    dataset['ALL_CNT'] = len(dataset['columns'])
    for var in dataset['columns']:
        found = tuple(REGEXP.findall(var[1]))
        fear[found] += 1
    for f in fear:
        d = dataset.copy()
        d['FEAR'] = str(f)
        d['FEAR_CNT'] = fear[f]
        f_datasets.append(d)

### Create Pandas Series for metadata

In [11]:
dataset_dict = {}
for dataset in f_datasets:
    for k in dataset:
        dataset_dict[k] = []
for dataset in f_datasets:
    for k in dataset_dict:
        if k in dataset:
            dataset_dict[k].append(dataset[k])
        else:
            dataset_dict[k].append(None)
dataset_pd = pd.DataFrame.from_dict(data=dataset_dict)
dataset_pd['_id'] = dataset_pd['_id'].apply(lambda i: str(i))

In [13]:
dataset_pd[['_id', 'FEAR', 'FEAR_CNT', 'ALL_CNT']].head()

,_id,FEAR,FEAR_CNT,ALL_CNT
0,601445d18067262630c27f98,"(u'log', u'naive latest value')",1,2
1,601445d18067262630c27f98,"(u'actual',)",1,2
2,601445d18067262630c27f9e,"(u'average', u'log')",1,117
3,601445d18067262630c27f9e,"(u'log', u'naive latest value')",1,117
4,601445d18067262630c27f9e,"(u'No History Available',)",1,117


### Create set of TOP 5 models IDs from all leaderboards

In [14]:
top_5 = set()
models_has_codegen = {}
top_dict = defaultdict(int)
for index, row in sorted_models.iterrows():
    top_dict[row['pid']] += 1
    models_has_codegen[row['lid']] = row['has_codegen']
    if top_dict[row['pid']] <= 5:
        top_5.add(row['lid'])
len(top_5)

505

### Aggregate the data and prepare Pandas dataset for the final stats

In [15]:
fear_stats = []
for index, row in dataset_pd.iterrows():
    x = {
        'FEAR': row['FEAR'],
        'FEAR_CNT': row['FEAR_CNT'],
        'ALL_CNT': row['ALL_CNT'],
    }
    m = sorted_models.loc[sorted_models.dataset_id == row['_id']]
    lid = m.lid.values[0]
    p = project_pd.loc[project_pd._id == str(m.pid.values[0])]
    
    x['IN_TOP_5'] = lid in top_5
    x['HAS_CODEGEN'] = bool(models_has_codegen[lid])
    x['MODEL'] = m.model_type.values[0]
    x['DATASET_NAME'] = row['name']
    x['PROJECT_NAME'] = p.project_name.values[0]
    fear_stats.append(x)
fear_dict = defaultdict(list)
for fear in fear_stats:
    for k in fear:
        fear_dict[k].append(fear[k])
fear_pd = pd.DataFrame.from_dict(data=fear_dict)
fear_pd.head()

,ALL_CNT,DATASET_NAME,FEAR,FEAR_CNT,HAS_CODEGEN,IN_TOP_5,MODEL,PROJECT_NAME
0,2,Baseline Only (latest),"(u'log', u'naive latest value')",1,False,False,Baseline Predictions Using Most Recent Value,datarobot-private-datasets-nonredistributable/...
1,2,Baseline Only (latest),"(u'actual',)",1,False,False,Baseline Predictions Using Most Recent Value,datarobot-private-datasets-nonredistributable/...
2,117,No Differencing,"(u'average', u'log')",1,False,True,AVG Blender,datarobot-private-datasets-nonredistributable/...
3,117,No Differencing,"(u'log', u'naive latest value')",1,False,True,AVG Blender,datarobot-private-datasets-nonredistributable/...
4,117,No Differencing,"(u'No History Available',)",1,False,True,AVG Blender,datarobot-private-datasets-nonredistributable/...


In [16]:
fear_pd.to_csv('mbtest_fear_stats_601426c300e8776eebb30424.csv')

/home/tony.hurkovskyi/.virtualenvs/datarobot-7.0/lib/python2.7/site-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in a future release.
  from cryptography import x509


# Prepare General Stats supporting of the Codegen

### the logic below could take a lot of time, so it needs to be refined and invoked carefully!!!!

In [17]:
projects = []
for pid in set(data.pid.values):
    models_cursor = db.leaderboard.find({'pid': ObjectId(pid)})
    models = [model for model in models_cursor]
    if not models:
        continue
    project = db.project.find_one({'_id': ObjectId(pid)})
    sorted_models = sorted(models, key=lambda m: m['test'][project['metric']])[:2]
    info_models = [{
        'lid': m['lid'],
        'project_name': project['project_name'],
        'vertices': m.get('task_version', {}).keys(),
        'dataset_id': m['dataset_id'],
        'model_type': m['model_type'],
        'has_codegen': m.get('has_codegen', False)
        } 
        for m in sorted_models]
    dataset_ids = {m['dataset_id'] for m in info_models}
    dataset_dict = {i: db.metadata.find_one({'_id': ObjectId(i)}) for i in dataset_ids}
    dataset_dict_col = {}
    for k in dataset_dict:
        fear_set = set()
        for var in dataset_dict[k]['columns']:
            found = tuple(REGEXP.findall(var[1]))
            if found:
                fear_set.add(found)
        dataset_dict_col[k] = fear_set
    for info in info_models:
        info['fear'] = dataset_dict_col[info['dataset_id']]
        
    projects += info_models

## Prepare data to be used by Pandas

In [18]:
mydict = {
    'project_name': [p['project_name'] for p in projects],
    'fear': [list(p['fear']) for p in projects],
    'has_codegen': [p['has_codegen'] for p in projects],
    'model_type': [p['model_type'] for p in projects],
    'vertices': [p['vertices'] for p in projects]
}

In [19]:
pd.DataFrame.from_dict(data=mydict).to_csv('mbtest_stats_601426c300e8776eebb30424.csv')